# 🔍 Azure Document Extraction Pipeline

This notebook demonstrates how to extract structured data from documents using:

1. **Mistral Document AI** - OCR and text extraction
2. **GPT-5.1 (Responses API)** - Structured extraction with strict JSON Schema
3. **Azure AI Evaluation SDK** - Quality evaluation (optional)

## Architecture

```
📄 Document (PDF/Image)
       │
       ▼
┌──────────────────────┐
│  Mistral Document AI │  ← OCR
└──────────────────────┘
       │
       ▼ Markdown Text
┌──────────────────────┐
│  GPT-5.1             │  ← Structured extraction
│  (Responses API)     │     Strict JSON Schema
└──────────────────────┘
       │
       ▼ JSON Data
┌──────────────────────┐
│  Azure AI Evaluation │  ← Quality (optional)
└──────────────────────┘
```

## 1️⃣ Installation

Install dependencies (once):

In [1]:
# [Cell 01] Install dependencies
# Uncomment to install
# !pip install -r ../requirements.txt

## 2️⃣ Configuration

Before continuing:
1. Copy `.env.example` to `.env` in the root folder
2. Fill in your Azure endpoints and keys
3. Log in to Azure: `az login`

In [2]:
# [Cell 02] Load configuration
import sys
sys.path.insert(0, "../src")

from config import Config

config = Config.from_env("../.env")
config.show_status()

Configuration Status
✅ Mistral Document AI
   Endpoint: https://felat-mgtjiioy-francecentral.cognitiveserv...
   Model: mistral-document-ai-2505-2

✅ Azure OpenAI (Responses API)
   Endpoint: https://doc-processing-mistral-gpt-eval.cognitives...
   Base URL: https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com/openai/v1/
   Deployment: gpt-5.1

✅ Evaluation: Configured
   Deployment: gpt-4o


{'mistral_configured': True, 'aoai_configured': True, 'eval_configured': True}

## 3️⃣ Create the Pipeline

The pipeline automatically initializes all components:

In [3]:
# [Cell 03] Create pipeline
from pipeline import DocumentPipeline

pipeline = DocumentPipeline(config)

🔧 Initializing pipeline components...
   ✅ OCR client ready (model: mistral-document-ai-2505-2)
   ✅ Extractor ready (model: gpt-5.1)
   ✅ Evaluator ready (model: gpt-4o)
✅ Pipeline ready



## 4️⃣ Process a Document

If your document is at the project root (e.g., `invoice.png`), use the path `../invoice.png` from this notebook (located in `notebooks/`).

In [4]:
# [Cell 04] Process document
results = pipeline.process(
    file_path="../invoice.png",
    verbose=True
)

📄 Processing: invoice.png
   Size: 0.353 MB

🔄 Step 1: OCR with Mistral Document AI
   ✅ OCR complete (1 pages)

🔄 Step 2: Structured extraction with GPT-5.1
   ✅ Extraction complete (confidence: 0.96)

✅ Processing complete


## 5️⃣ Display Results

In [5]:
# [Cell 05] Display results (formatted)
pipeline.display_results(results)


📊 EXTRACTION RESULTS

Document Type: invoice
Document Number: 0324
Document Date: None
Total Amount: 500 USD
Confidence: 0.96

Line Items (5):
   1. Sed ut perspiciatis unde omnis iste lore... → 100
   2. Adipisc ing elit, sed do eius mod tempor... → 100
   3. Nemo enim ipsam volupt atem quia volupta... → 100
   4. Sed ut perspiciatis unde omnis iste lore... → 100
   5. Adipisc ing elit, sed do eius mod tempor... → 100


In [6]:
# [Cell 06] Display results (full JSON)
import json
print(json.dumps(results["extraction"], indent=2, ensure_ascii=False))

{
  "document_type": "invoice",
  "document_number": "0324",
  "document_date": null,
  "supplier": {
    "name": "Paul J Gordon",
    "address": "2758 Paul Wayne Haggerty Road Kenner, LA 70062",
    "tax_id": null
  },
  "customer": null,
  "total_amount": {
    "amount": 500,
    "currency": "USD"
  },
  "line_items": [
    {
      "description": "Sed ut perspiciatis unde omnis iste lorem",
      "quantity": 5,
      "unit_price": 50,
      "total": 100
    },
    {
      "description": "Adipisc ing elit, sed do eius mod tempor",
      "quantity": 2,
      "unit_price": 50,
      "total": 100
    },
    {
      "description": "Nemo enim ipsam volupt atem quia voluptas",
      "quantity": 5,
      "unit_price": 50,
      "total": 100
    },
    {
      "description": "Sed ut perspiciatis unde omnis iste lorem",
      "quantity": 7,
      "unit_price": 50,
      "total": 100
    },
    {
      "description": "Adipisc ing elit, sed do eius mod tempor",
      "quantity": 2,
      "unit_p

## 6️⃣ With Quality Evaluation (Optional)

Enable `run_evaluation=True` to measure extraction quality:

In [7]:
# [Cell 07] Process with evaluation
results_with_eval = pipeline.process(
    file_path="../invoice.png",
    run_evaluation=True,  # Enable evaluation
    verbose=True
)

pipeline.display_results(results_with_eval)

📄 Processing: invoice.png
   Size: 0.353 MB

🔄 Step 1: OCR with Mistral Document AI
   ✅ OCR complete (1 pages)

🔄 Step 2: Structured extraction with GPT-5.1
   ✅ Extraction complete (confidence: 0.96)

🔄 Step 3: Quality evaluation
   ✅ Evaluation complete (score: 89.6%)

✅ Processing complete

📊 EXTRACTION RESULTS

Document Type: invoice
Document Number: 0324
Document Date: None
Total Amount: 500 USD
Confidence: 0.96

Line Items (5):
   1. Sed ut perspiciatis unde omnis iste lore... → 100
   2. Adipisc ing elit, sed do eius mod tempor... → 100
   3. Nemo enim ipsam volupt atem quia volupta... → 100
   4. Sed ut perspiciatis unde omnis iste lore... → 100
   5. Adipisc ing elit, sed do eius mod tempor... → 100

📈 EVALUATION RESULTS
   ✅ Groundedness: 5.0/5
   ✅ Relevance: 4.0/5
   ✅ Coherence: 4.0/5

Validation: 3/3 checks passed

✅ OVERALL SCORE: 89.6%


## 7️⃣ Direct PDF Mode (Skip OCR)

For PDFs, you can use the Responses API's native PDF support:

In [8]:
# [Cell 08] Direct PDF mode (skip OCR)
# results = pipeline.process(
#     file_path="../examples/document.pdf",
#     use_direct_pdf=True,  # Use native PDF support
#     verbose=True
# )

## 8️⃣ Custom Schema

Create your own Pydantic schema for specific document types:

In [9]:
# [Cell 09] Define custom schema
from pydantic import BaseModel, Field
from typing import Optional, List

class ReceiptExtraction(BaseModel):
    """Schema for receipts."""
    
    store_name: str = Field(description="Store name")
    store_address: Optional[str] = Field(default=None, description="Address")
    date: Optional[str] = Field(default=None, description="Date (YYYY-MM-DD)")
    items: List[str] = Field(default_factory=list, description="List of items")
    subtotal: Optional[float] = Field(default=None, description="Subtotal")
    tax: Optional[float] = Field(default=None, description="Tax")
    total: float = Field(description="Total")
    payment_method: Optional[str] = Field(default=None, description="Payment method")
    confidence_score: float = Field(description="Confidence score (0-1)")

print("✅ ReceiptExtraction schema defined")

✅ ReceiptExtraction schema defined


In [10]:
# [Cell 10] Use custom schema
# results = pipeline.process_with_schema(
#     file_path="../examples/receipt.png",
#     schema_model=ReceiptExtraction,
#     verbose=True
# )

## 9️⃣ Export Results

In [11]:
# [Cell 11] Export results
pipeline.export_results(results, "../output/extraction_results.json")

✅ Results exported to: ../output/extraction_results.json


## 📚 Going Further

### Customization

- **Schemas**: Edit `src/schemas.py` to add your own models
- **Instructions**: Use `custom_instructions` to guide extraction
- **Validation**: Add rules in `src/evaluator.py`

### Documentation

- [Azure OpenAI Responses API](https://learn.microsoft.com/azure/ai-services/openai/how-to/responses)
- [Structured Outputs](https://learn.microsoft.com/azure/ai-services/openai/how-to/structured-outputs)
- [Azure AI Evaluation SDK](https://learn.microsoft.com/azure/ai-studio/how-to/evaluate-sdk)